<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/%CE%95%CF%81%CE%B3%CE%B1%CF%83%CF%84%CE%B7%CF%81%CE%B9%CE%B1%CE%BA%CE%AE_%CE%86%CF%83%CE%BA%CE%B7%CF%83%CE%B7_%CE%A4%CE%B5%CF%87%CE%BD%CE%B9%CE%BA%CE%AD%CF%82_%CE%94%CE%B9%CE%B1%CF%87%CE%B5%CE%AF%CF%81%CE%B9%CF%83%CE%B7%CF%82_%CE%BA%CE%B1%CE%B9_%CE%95%CE%BE%CF%8C%CF%81%CF%85%CE%BE%CE%B7%CF%82_%CE%B3%CE%B9%CE%B1_%CE%94%CE%B5%CE%B4%CE%BF%CE%BC%CE%AD%CE%BD%CE%B1_%CE%9C%CE%B5%CE%B3%CE%AC%CE%BB%CE%BF%CF%85_%CE%8C%CE%B3%CE%BA%CE%BF%CF%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import sparknlp

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[4]") \
    .config("spark.driver.memory","16G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.1") \
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_train.csv", inferSchema=True, header=True)
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_labels.csv", inferSchema=True, header=True)
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_test.csv", inferSchema=True, header=True)

# Show the first few rows of the DataFrame
data.show()


In [ ]:
print(data.columns)


In [ ]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner

# Assemble the text column into a Document type column
document_assembler = DocumentAssembler() \
    .setInputCol("medical_abstract") \
    .setOutputCol("document")

# Tokenize the document
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize the tokens (remove punctuation and convert to lowercase)
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# Remove stopwords
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("clean_tokens") \
    .setCaseSensitive(False)

# Finish the preprocessing (convert the result back to a string column)
finisher = Finisher() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCols(["clean_text"]) \
    .setOutputAsArray(False) \
    .setCleanAnnotations(False)

# Define the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


In [ ]:
from sparknlp.annotator import WordEmbeddingsModel

# Load the pre-trained word embeddings model
embeddings = WordEmbeddingsModel.pretrained('glove_100d', 'en') \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

# Add the embeddings to the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, embeddings, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          embeddings|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{word_embeddings...|Obstructive@sleep...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{word_embeddings...|Neutrophil@functi...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0, 0, A,..

In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Create a SentenceEmbeddings object
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence_embeddings to the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, embeddings, sentence_embeddings, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          embeddings| sentence_embeddings|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{word_embeddings...|[{sentence_embedd...|Obstructive@sleep...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{word_embeddings...|[{sentence_embedd...|Neutrophil@functi...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0,

In [33]:
train_data, test_data = data_clean.randomSplit([0.8, 0.2], seed=12345)
